In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
# Analysis and Holdings
earnings_estimate = pd.read_csv('data/analysis_and_holdings/AAPL_earnings_estimate.csv')
earnings_history = pd.read_csv('data/analysis_and_holdings/AAPL_earnings_history.csv')
eps_revisions = pd.read_csv('data/analysis_and_holdings/AAPL_eps_revisions.csv')
eps_trend = pd.read_csv('data/analysis_and_holdings/AAPL_eps_trend.csv')
growth_estimates = pd.read_csv('data/analysis_and_holdings/AAPL_growth_estimates.csv')
insider_purchases = pd.read_csv('data/analysis_and_holdings/AAPL_insider_purchases.csv')
insider_roster_holders = pd.read_csv('data/analysis_and_holdings/AAPL_insider_roster_holders.csv')
institutional_holders = pd.read_csv('data/analysis_and_holdings/AAPL_institutional_holders.csv')
major_holders = pd.read_csv('data/analysis_and_holdings/AAPL_major_holders.csv')
mutualfunds_holders = pd.read_csv('data/analysis_and_holdings/AAPL_mutualfund_holders.csv')
reccomendations = pd.read_csv('data/analysis_and_holdings/AAPL_recommendations.csv')
revenue_estimate = pd.read_csv('data/analysis_and_holdings/AAPL_revenue_estimate.csv')
sustainability = pd.read_csv('data/analysis_and_holdings/AAPL_sustainability.csv')
upgrades_downgrades = pd.read_csv('data/analysis_and_holdings/AAPL_upgrades_downgrades.csv')

# Financials
balance_sheet = pd.read_csv('data/financials/AAPL_balance_sheet.csv', index_col=0)
cashflow = pd.read_csv('data/financials/AAPL_cashflow.csv', index_col=0)
income_statement = pd.read_csv('data/financials/AAPL_income_statement.csv', index_col=0)

# Info
actions = pd.read_csv('data/info/AAPL_actions.csv', index_col=0)
capital_gains = pd.read_csv('data/info/AAPL_capital_gains.csv', index_col=0)
dividends = pd.read_csv('data/info/AAPL_dividends.csv', index_col=0)
fast_info = pd.read_csv('data/info/AAPL_fast_info.csv', index_col=0)
history = pd.read_csv('data/info/AAPL_history.csv', index_col=0)
info = pd.read_csv('data/info/AAPL_info.csv')
splits = pd.read_csv('data/info/AAPL_splits.csv', index_col=0)

# Price and Volume
ohlc = pd.read_csv('data/price/AAPL_price_volume.csv', index_col=0, skiprows=2)
vix = pd.read_csv('../VIX/data/price/VIX_price_volume.csv', header=None, skiprows=3)

# Macro data
to_year_treasury_index = pd.read_csv('../market_data/data_transformed/10_year_treasury_index.csv')
dow_jones = pd.read_csv('../market_data/data_transformed/Dow_Jones_index.csv')
nasdaq = pd.read_csv('../market_data/data_transformed/Nasdaq_index.csv')
nyse_composite = pd.read_csv('../market_data/data_transformed/NYSE_composite_index.csv')
oil = pd.read_csv('../market_data/data_transformed/Oil_index.csv')
phlx_semiconductor_index = pd.read_csv('../market_data/data_transformed/phlx_semiconductor_index.csv')
russell_200 = pd.read_csv('../market_data/data_transformed/russell2000.csv')
sp_500 = pd.read_csv('../market_data/data_transformed/sp500.csv')

# Data preprocessing

In [4]:
numeric_cols = ['Open', 'High', 'Low', 'Close']

ohlc.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

for col in numeric_cols:
    ohlc[col] = pd.to_numeric(ohlc[col], errors='coerce')

In [5]:
ohlc['Volume'] = pd.to_numeric(ohlc['Volume'], errors='coerce')
ohlc['High'] = pd.to_numeric(ohlc['High'], errors='coerce')
ohlc['Low'] = pd.to_numeric(ohlc['Low'], errors='coerce')
ohlc['Open'] = pd.to_numeric(ohlc['Open'], errors='coerce')
ohlc['Close'] = pd.to_numeric(ohlc['Close'], errors='coerce')


all_time_low = ohlc['Low'].min()
all_time_low_date = ohlc['Low'].idxmin()
print(f'All time low: {all_time_low} on {all_time_low_date}')

all_time_high = ohlc['High'].max()
all_time_high_date = ohlc['High'].idxmax()
print(f'All time high: {all_time_high} on {all_time_high_date}')

all_time_low_vol = ohlc['Volume'].min()
all_time_low_vol_date = ohlc['Volume'].idxmin()
print(f'All time low volume: {all_time_low_vol} on {all_time_low_vol_date}')

all_time_high_vol = ohlc['Volume'].max()
all_time_high_vol_date = ohlc['Volume'].idxmax()
print(f'All time high volume: {all_time_high_vol} on {all_time_high_vol_date}')

All time low: 0.037681121379137 on 1982-07-08
All time high: 259.239990234375 on 2025-10-03
All time low volume: 0 on 1981-08-10
All time high volume: 7421640800 on 2000-09-29


In [6]:
data = ohlc.copy()

data.index = pd.to_datetime(data.index, format='%Y-%m-%d')

Adding percentage change of the closing price# Feature Engineering

In [7]:
data['Return'] = data['Close'].pct_change()

Adding volatility indicators

In [8]:
data['Volatility_20'] = data['Close'].rolling(window=20).std()
data['Volatility_50'] = data['Close'].rolling(window=50).std()

Adding ATR (Average True Range) — captures volatility from high/low ranges

In [9]:
high_low = data['High'] - data['Low']
high_close = (data['High'] - data['Close'].shift()).abs()
low_close = (data['Low'] - data['Close'].shift()).abs()

data['TR'] = high_low.combine(high_close, max).combine(low_close, max)
data['ATR_14'] = data['TR'].rolling(window=14).mean()

Adding moving averages

In [10]:
windows = [5, 20, 50, 200]

for w in windows:
    data[f'SMA_{w}'] = data['Close'].rolling(window=w).mean()
    data[f'EMA_{w}'] = data['Close'].ewm(span=w, adjust=False).mean()

Momentum indicators

In [11]:
# RSI (Relative Strength Index)
delta = data['Close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)

avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()

rs = avg_gain / avg_loss
data['RSI_14'] = 100 - (100 / (1 + rs))

# MACD (Moving Average Convergence Divergence)
ema_12 = data['Close'].ewm(span=12, adjust=False).mean()
ema_26 = data['Close'].ewm(span=26, adjust=False).mean()

data['MACD'] = ema_12 - ema_26
data['MACD_signal'] = data['MACD'].ewm(span=9, adjust=False).mean()

# Stochastic Oscillator
low_14 = data['Low'].rolling(14).min()
high_14 = data['High'].rolling(14).max()

data['Stochastic'] = 100 * (data['Close'] - low_14) / (high_14 - low_14)

# ADX (Average Directional Index)
high = data['High']
low = data['Low']
close = data['Close']

plus_dm = high.diff()
minus_dm = low.diff().abs()
plus_dm[plus_dm < 0] = 0
minus_dm[minus_dm < 0] = 0

tr = pd.concat([high - low, (high - close.shift()).abs(), (low - close.shift()).abs()], axis=1).max(axis=1)
atr = tr.rolling(14).mean()

plus_di = 100 * (plus_dm.rolling(14).sum() / atr)
minus_di = 100 * (minus_dm.rolling(14).sum() / atr)

dx = 100 * (abs(plus_di - minus_di) / (plus_di + minus_di))
data['ADX_14'] = dx.rolling(14).mean()

# On-Balance Volume (OBV)
data['OBV'] = (np.sign(data['Close'].diff()) * data['Volume']).fillna(0).cumsum()

# Money Flow Index (MFI) - fully vectorized
typical_price = (data['High'] + data['Low'] + data['Close']) / 3
money_flow = typical_price * data['Volume']
tp_diff = typical_price.diff()

positive_mf = money_flow.where(tp_diff > 0, 0).rolling(14).sum()
negative_mf = money_flow.where(tp_diff < 0, 0).rolling(14).sum()
data['MFI_14'] = 100 * (positive_mf / (positive_mf + negative_mf))

# Basic Candlestick Patterns
data['Doji'] = abs(data['Close'] - data['Open']) <= 0.1 * (data['High'] - data['Low'])
data['Hammer'] = ((data['High'] - data['Low']) > 3 * (data['Open'] - data['Close'])) & \
                 ((data['Close'] - data['Low']) / (0.001 + data['High'] - data['Low']) > 0.6)
data['Engulfing'] = ((data['Close'] > data['Open'].shift()) &
                     (data['Open'] < data['Close'].shift()) &
                     (data['Close'] - data['Open'] > data['Open'].shift() - data['Close'].shift()))

Fundamental features

Dividends (0/1)

In [12]:
# Make sure data index is timezone-naive
data.index = pd.to_datetime(data.index).tz_localize(None)

# Convert dividends index to datetime (force conversion and make tz-naive safely)
dividend_dates = pd.to_datetime(dividends.index, utc=True, errors='raise')  # convert all to UTC first
dividend_dates = dividend_dates.tz_convert(None)  # then drop tz info

# Remove any invalid (NaT) entries
dividend_dates = dividend_dates.dropna()

# Compare only by date (normalize removes the time part)
dividend_dates = set(dividend_dates.normalize())

# Create Dividends (0/1) column in data
data['Dividends'] = data.index.normalize().isin(dividend_dates).astype(int)

Split (0/1)

In [13]:
splits_dates = pd.to_datetime(splits.index, utc=True, errors='raise')
splits_dates = splits_dates.tz_convert(None)

splits_dates = splits_dates.dropna()

splits_dates = set(splits_dates.normalize())

data['Splits'] = data.index.normalize().isin(splits_dates).astype(int)

Macro context

VIX

The VIX (Volatility Index) is often called the “fear gauge” of the market. It represents the market’s expectation of 30-day forward-looking volatility in the S&P 500 index, derived from options prices.
* High VIX → Market expects high volatility, often during market stress or uncertainty.
* Low VIX → Market expects low volatility, generally in calm or bullish periods.
* VIX is expressed in percentage points (e.g., VIX = 20 means the market expects ~20% annualized volatility over the next 30 days).

Important: VIX is not a price index; it’s a measure of expected volatility.

In [15]:
vix.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
vix['Date'] = pd.to_datetime(vix['Date'])
vix.set_index('Date', inplace=True)

vix.drop(columns=['Volume'], inplace=True)

data = data.join(vix, how='left', rsuffix='_VIX')

In [16]:
# VIX returns
data['VIX_Return'] = data['Close_VIX'].pct_change()

# Volatility
data['VIX_Volatility_20'] = data['Close_VIX'].rolling(window=20).std()
data['VIX_Volatility_50'] = data['Close_VIX'].rolling(window=50).std()

# Trend
data['VIX_SMA_20'] = data['Close_VIX'].rolling(window=20).mean()
data['VIX_EMA_20'] = data['Close_VIX'].ewm(span=20, adjust=False).mean()
data['VIX_RollingVol'] = data['Close_VIX'].rolling(5).std()
data['VIX_Change'] = data['Close_VIX'] - data['Close_VIX'].shift(1)

# Lagged features
for lag in range(1, 6):
    data[f'VIX_Close_lag{lag}'] = data['Close_VIX'].shift(lag)